# Character Recognition

## Importing and data set up

In [1]:
%matplotlib inline

from utils import *

path = '../data/nietzsche/'

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
text = open(path+'nietzsche.txt')

In [4]:
text=text.read()

In [13]:
len(text)

600893

In [19]:
chars = sorted(list(set(text)))

In [21]:
vocab_size=len(chars)+1
print('Total characters : ',vocab_size)

Total characters :  85


In [22]:
chars.insert(0,'\0')

In [26]:
char_indices = {c:i for i,c in enumerate(chars)}
indices_char = {i:c for i,c in enumerate(chars)}

In [28]:
idx=[char_indices[c] for c in text]

In [33]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 char model

In [38]:
cs=3

c1_dat = [idx[i] for i in range(0,len(idx)-1-cs,cs)]
c2_dat = [idx[i+1] for i in range(0,len(idx)-1-cs,cs)]
c3_dat = [idx[i+2] for i in range(0,len(idx)-1-cs,cs)]
c4_dat = [idx[i+3] for i in range(0,len(idx)-1-cs,cs)]

In [45]:
''.join(indices_char[i] for i in c1_dat[:10])

'PFE\nPSGh u'

In [52]:
x1=np.stack(c1_dat[:-2])
x2=np.stack(c2_dat[:-2])
x3=np.stack(c3_dat[:-2])

In [54]:
y=np.stack(c4_dat[:-2])

In [59]:
n_fac=42

In [61]:
x1[:4],x2[:4],x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [66]:
max(y)

83

In [67]:
def embedding_input(name,n_in,n_out):
    inp=Input(shape=(1,),dtype='int64',name=name)
    emb=Embedding(n_in,n_out,input_length=1)(inp)
    return inp,Flatten()(emb)

In [68]:
c1_in,c1=embedding_input('c1',vocab_size,n_fac)
c2_in,c2=embedding_input('c2',vocab_size,n_fac)
c3_in,c3=embedding_input('c3',vocab_size,n_fac)

In [70]:
c1

<tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>

In [71]:
n_hidden= 256

In [72]:
dense_in = Dense(n_hidden,activation='relu')

In [73]:
c1_hidden=dense_in(c1)

In [74]:
dense_hidden = Dense(n_hidden,activation='relu')

In [75]:
c2_dense=dense_in(c2)

In [77]:
hidden_2=dense_hidden(c1_hidden)
c2_hidden = keras.layers.add([c2_dense,hidden_2])

In [79]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = keras.layers.add([c3_dense, hidden_3])

In [80]:
dense_out = Dense(vocab_size, activation='softmax')

In [81]:
c4_out = dense_out(c3_hidden)

In [82]:
model=Model([c1_in,c2_in,c3_in],c4_out)

In [83]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [84]:
model.optimizer.lr=0.000001

In [85]:
model.fit([x1,x2,x3],y,batch_size=64,epochs=4)

Epoch 1/4
200295/200295 [==============================] - 14s - loss: 4.4225    
Epoch 2/4
200295/200295 [==============================] - 13s - loss: 4.3512    
Epoch 3/4
200295/200295 [==============================] - 13s - loss: 4.1987    
Epoch 4/4
200295/200295 [==============================] - 13s - loss: 3.9123    


In [86]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c3 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 42)         3570        c3[0][0]                         
___________________________________________________________________________________________

In [87]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [88]:
get_next(' th')

' '

In [91]:
model.optimizer.lr=0.01

model.fit([x1,x2,x3],y,batch_size=64,epochs=4)

In [93]:
model.optimizer.lr=0.000001

In [94]:
model.fit([x1,x2,x3],y,batch_size=64,epochs=4)

Epoch 1/4
200295/200295 [==============================] - 14s - loss: 3.1020    
Epoch 2/4
200295/200295 [==============================] - 14s - loss: 3.0900    
Epoch 3/4
200295/200295 [==============================] - 14s - loss: 3.0803    
Epoch 4/4
200295/200295 [==============================] - 14s - loss: 3.0718    


In [95]:
get_next(' th')

' '

In [96]:
model.optimizer.lr=0.01

model.fit([x1,x2,x3],y,batch_size=64,epochs=4)

Epoch 1/4
200295/200295 [==============================] - 14s - loss: 3.0638    
Epoch 2/4
200295/200295 [==============================] - 14s - loss: 3.0561    
Epoch 3/4
200295/200295 [==============================] - 13s - loss: 3.0482    
Epoch 4/4
200295/200295 [==============================] - 14s - loss: 3.0403    


In [98]:
get_next(' an')

' '

## OUR FIRST RNN

In [103]:
cs=8

In [106]:
c_in_dat = [[idx[i+n] for i in range(0,len(idx)-1-cs,cs)] for n in range(cs)]

In [109]:
np.shape(c_in_dat)

(8, 75111)

In [110]:
c_out_dat = [idx[i+cs] for i in range(0,len(idx)-1-cs,cs)]

In [111]:
np.shape(c_out_dat)

(75111,)

In [112]:
xs=[np.stack(c[:-2]) for c in c_in_dat]

In [128]:
y=np.stack(c_out_dat[:-2])

In [114]:
len(xs),xs[0].shape

(8, (75109,))

In [132]:
np.shape(xs)

(8, 75109)

In [130]:
y.shape

(75109,)

In [127]:
[print(indices_char[x1[i]],indices_char[x2[i]],indices_char[x3[i]],indices_char[y[i]]) for i in range(20,30)]


r e   n
n o t  
  g r o
o u n d
d 
 f o
o r   s
s u s p
p e c t
t i n g
g   t h


[None, None, None, None, None, None, None, None, None, None]

In [133]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [135]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [136]:
n_fac=42

In [145]:
[indices_char[xs[n][3]] for n in range(cs)]

[' ', 'T', 'r', 'u', 't', 'h', ' ', 'i']

In [146]:
indices_char[y[3]]

's'

### Create and Train model

In [149]:
def embedding_input(name,n_in,n_out):
    inp=Input(shape=(1,),dtype='int64',name=name+'_in')
    emb=Embedding(n_in,n_out,input_length=1,name=name+'_emb')(inp)
    return inp,Flatten()(emb)

In [151]:
c_ins=[embedding_input('c'+str(n),vocab_size,n_fac) for n in range(cs)]

In [152]:
n_hidden=256

In [155]:
dense_in=Dense(n_hidden,activation='relu')
dense_hidden=Dense(n_hidden,activation='relu',kernel_initializer='identity')
dense_out=Dense(vocab_size,activation='softmax')

In [157]:
hidden=dense_in(c_ins[0][1])

In [159]:
for i in range(1,cs):
    c_dense=dense_in(c_ins[i][1])
    hidden=dense_hidden(hidden)
    hidden=keras.layers.add([c_dense,hidden])

In [160]:
c_out=dense_out(hidden)

In [161]:
model=Model([c[0] for c in c_ins],c_out)
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [166]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c7_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c6_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c5_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c4_in (InputLayer)               (None, 1)             0                                            
___________________________________________________________________________________________

In [167]:
model.fit(xs,y,batch_size=64,epochs=12)

Epoch 1/12
75109/75109 [==============================] - 11s - loss: 2.5394    
Epoch 2/12
75109/75109 [==============================] - 10s - loss: 2.2534    
Epoch 3/12
75109/75109 [==============================] - 11s - loss: 2.1526    
Epoch 4/12
75109/75109 [==============================] - 10s - loss: 2.0853    
Epoch 5/12
75109/75109 [==============================] - 11s - loss: 2.0266    
Epoch 6/12
75109/75109 [==============================] - 11s - loss: 1.9801    
Epoch 7/12
75109/75109 [==============================] - 10s - loss: 1.9408    
Epoch 8/12
75109/75109 [==============================] - 10s - loss: 1.9073    
Epoch 9/12
75109/75109 [==============================] - 10s - loss: 1.8732    
Epoch 10/12
75109/75109 [==============================] - 11s - loss: 1.8463    
Epoch 11/12
75109/75109 [==============================] - 10s - loss: 1.8212    
Epoch 12/12
75109/75109 [==============================] - 10s - loss: 1.7972    


In [168]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [169]:
get_next('for thos')

'e'

In [170]:
get_next('part of ')

't'

In [171]:
get_next('queens a')

'n'

## OUR First RNN with Keras

In [175]:
n_hidden,n_fac,cs,vocab_size=(256,42,8,86)

In [177]:
model = Sequential([
    Embedding(vocab_size,n_fac,input_length=cs),
    SimpleRNN(n_hidden,activation='relu',recurrent_initializer='identity'),
    Dense(vocab_size,activation='softmax')
])

In [178]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_13 (Dense)             (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [179]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [185]:
model.fit(np.concatenate(xs,axis=1),y,batch_size=64,epochs=8)

Epoch 1/8
75109/75109 [==============================] - 12s - loss: 2.7925    
Epoch 2/8
75109/75109 [==============================] - 11s - loss: 2.2711    
Epoch 3/8
75109/75109 [==============================] - 11s - loss: 2.0655    
Epoch 4/8
75109/75109 [==============================] - 11s - loss: 1.9286    
Epoch 5/8
75109/75109 [==============================] - 11s - loss: 1.8258    
Epoch 6/8
75109/75109 [==============================] - 11s - loss: 1.7440    
Epoch 7/8
75109/75109 [==============================] - 11s - loss: 1.6798    
Epoch 8/8
75109/75109 [==============================] - 11s - loss: 1.6286    


In [186]:
idxs=[char_indices[c] for c in 'for thos']

In [187]:
idxs

[59, 68, 71, 2, 73, 61, 68, 72]

In [188]:
arrs=np.array(idxs)[np.newaxis,:]

In [192]:
np.array(idxs)[0]

59

In [195]:
arrs[0]

array([59, 68, 71,  2, 73, 61, 68, 72])

In [196]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [197]:
get_next_keras('part of ')

't'

In [198]:
get_next_keras('queens a')

'n'

## Returning Sequences

In [200]:
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [205]:
ys=[np.stack(c[:-2]) for c in c_out_dat]

In [206]:
np.shape(ys)

(8, 75109)

### Sequence model with Keras

In [208]:
n_hidden,n_fac,cs,vocab_size

(256, 42, 8, 86)

In [209]:
model = Sequential([
    Embedding(vocab_size,n_fac,input_length=cs),
    SimpleRNN(n_hidden,activation='relu',return_sequences=True,recurrent_initializer='identity'),
    TimeDistributed(Dense(vocab_size,activation='softmax'))
])

In [210]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [211]:
xs[0].shape

(75109, 1)

In [212]:
x_rnn=np.stack(np.squeeze(xs),axis=1)

In [215]:
y_rnn=np.atleast_3d(np.stack(ys,axis=1))

In [216]:
y_rnn.shape

(75109, 8, 1)

In [217]:
model.fit(x_rnn,y_rnn,epochs=8,batch_size=64)

Epoch 1/8
75109/75109 [==============================] - 17s - loss: 2.4376    
Epoch 2/8
75109/75109 [==============================] - 17s - loss: 1.9964    
Epoch 3/8
75109/75109 [==============================] - 18s - loss: 1.8810    
Epoch 4/8
75109/75109 [==============================] - 18s - loss: 1.8217    
Epoch 5/8
75109/75109 [==============================] - 17s - loss: 1.7837    
Epoch 6/8
75109/75109 [==============================] - 17s - loss: 1.7572    
Epoch 7/8
75109/75109 [==============================] - 16s - loss: 1.7371    
Epoch 8/8
75109/75109 [==============================] - 16s - loss: 1.7212    


In [218]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [219]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 's', ' ']

## Using LSTM stateful model with Keras

In [220]:
bs=64

In [221]:
model = Sequential([
    Embedding(vocab_size,n_fac,input_length=cs,batch_input_shape=(bs,cs)),
    BatchNormalization(),
    LSTM(n_hidden,return_sequences=True,stateful=True),
    TimeDistributed(Dense(vocab_size,activation='softmax'))
])

In [222]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [223]:
mx=len(x_rnn)//bs*bs

In [224]:
mx

75072

In [225]:
model.fit(x_rnn[:mx],y_rnn[:mx],batch_size=bs,epochs=4,shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 59s - loss: 2.2553    
Epoch 2/4
75072/75072 [==============================] - 56s - loss: 1.9923    
Epoch 3/4
75072/75072 [==============================] - 56s - loss: 1.9151    
Epoch 4/4
75072/75072 [==============================] - 57s - loss: 1.8700    
